# Diabetes con redes neuronales

Carga el dataset Diabetes con Scikit-learn, crea una variable llamada ``X`` que contenga las variables independientes del dataset y una variable llamada ``y`` que contenga sus targets:

In [35]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)

In [36]:
assert X.shape == (442, 10)
assert y.shape == (442,)

Para poder evaluar de forma fiable los modelos, lo primero que se debería hacer es definir folds para cross-validation o separar una parte de los datos como conjunto de test, por ejemplo un 10% de la muestra. Guarda un 90% de los datos en las variables ``X_train`` e ``y_train`` y el 10% restante en ``X_test`` e ``y_test``. Usa ``random_state``=0 y ``shuffle``=True:

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)

In [38]:
assert X_train.shape == (397, 10)
assert y_train.shape == (397,)
assert X_test.shape == (45, 10)
assert y_test.shape == (45,)

A continuación hay que crear un primer modelo predictivo. Dado que el problema es de regresión, lo más básico es una regresión lineal. Usa el método de entrenamiento mediante descenso por gradiente. Estos método suele funcionar mejor con una estandarización previa de los datos, por lo que tendrás que construir un pipeline de procesamiento en el que la primera componente será un estandarizador y la segunda el modelo predictivo. Guarda este objeto en una variable llamada ``estimator``:

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

estimator = Pipeline([
    ('std', StandardScaler()),
    ('sgdr', SGDRegressor(random_state=0))
])

estimator.fit(X_train, y_train)

Pipeline(steps=[('std', StandardScaler()),
                ('sgdr', SGDRegressor(random_state=0))])

In [40]:
assert type(estimator) == Pipeline
assert len(estimator.steps) == 2

Ahora se puede entrenar el modelo con el conjunto de datos de entrenamiento y evaluarlo con el conjunto de datos de test:

In [41]:
estimator.fit(X_train, y_train)
lr_score = estimator.score(X_test, y_test)
print(f"{lr_score}")

0.3448985394999857


El resultado es mejorable, pero ya hay un modelo base contra el que comparar. ¿Serías capaz de programar el mismo pipeline utilizando un perceptrón en lugar de una regresión lineal? Configura la red neuronal de manera que sea equivalente al modelo lineal previo y guarda este objeto en una variable llamada ``estimator2``:

In [42]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

base_estimator = Pipeline([
    ("std", StandardScaler()),
    ("mlp", MLPRegressor(max_iter=1000, random_state=0))
])

parameters = {
    'mlp__hidden_layer_sizes': [(10,), (20, 10), (30, 20, 10)],
    'mlp__learning_rate_init': uniform(0, 1)
}

estimator2 = RandomizedSearchCV(base_estimator, parameters, n_iter=4, random_state=0, cv=5)

In [43]:
estimator2.fit(X_train, y_train)
mlp_score = estimator2.score(X_test, y_test)
print(f"{mlp_score}")

0.2247048111847647


In [44]:
import numpy as np
assert type(estimator2) == Pipeline
assert len(estimator2.steps) == 2
assert type(estimator2.steps[0][1]) == StandardScaler
assert type(estimator2.steps[1][1]) == MLPRegressor
assert np.abs(lr_score - mlp_score)/mlp_score < 0.01
np.testing.assert_almost_equal(lr_score, mlp_score, decimal=2, err_msg='Los scores difieren demasiado')

AssertionError: 

Para conseguir mejorar el resultado será necesario hacer una búsqueda de meta-parámetros. Normalmente la búsqueda de meta-parámetros se automatiza con métodos como la búsqueda en rejilla o la búsqueda aleatoria. Intenta mejorar el resultado realizando una búsqueda de los meta-parámetros de la red neuronal. Al utilizar arquitecturas más complejas, es probable que haya que aumentar el número máximo de iteraciones y activar el early stopping. Guarda el objeto en una variable llamada ``search``:

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('std', StandardScaler()),
    ('mlp', MLPRegressor(max_iter=1000, early_stopping=True, random_state=0))
])

parameters = {
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'mlp__learning_rate_init': [0.001, 0.01],
}

search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=5, scoring='r2')

In [46]:
search.fit(X_train, y_train)
print(search.best_params_)
score = search.score(X_test, y_test)
print(f"{score}")

{'mlp__hidden_layer_sizes': (50, 50), 'mlp__learning_rate_init': 0.01}
0.17719414653728383


In [47]:
assert mlp_score < score

AssertionError: 

# Regresion Lineal por el metodo de Ordinary Least Squares

Los modelos predictivos de Scikit-learn se definen como clases que tienen al menos cuatro métodos:
* ``__init__``: el constructor, recibe como argumentos los meta-parámetros del modelo y los almacena
* ``fit``: recibe como argumentos X e y, entrena el modelo (calcula sus parámetros) y lo devuelve.
* ``predict``: recibe como argumento X, calcula las predicciones y las devuelve.
* ``score``: recibe como argumentos X e y, realiza las predicciones sobre X y calcula con ellas el error de aproximación a y. Por defecto aplica la métrica ``r2_score`` a los modelos de regresión y la métrica ``accuracy`` a los de clasifiación.

Programa a continuación el método fit de la regresión lineal mediante el método de Ordinary Least Squares.

In [48]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score

class OLSLinearRegression(BaseEstimator, RegressorMixin):

    def __init__(self):
        pass

    def fit(self, X, y):
        if len(X.shape) == 1: X = X.reshape((-1, 1))
        X = np.hstack((np.ones((len(X), 1)), X))
        self.beta_ = np.linalg.inv(X.T @ X) @ X.T @ y
        return self

    def predict(self, X):
        if len(X.shape) == 1: X = X.reshape((-1, 1))
        X = np.hstack((np.ones((len(X), 1)), X))
        return X @ self.beta_

    def score(self, X, y):
        preds = self.predict(X)
        return r2_score(y, preds)

La regresión lineal con el método de OLS ya está disponible en Scikit-learn mediante la clase ``LinearRegression`` del submódulo ``linear_model``. Si has programado el método de OLS correctamente, debería producir los mismos resultados que ``LinearRegression`` sobre los datasets de regresión de California Housing y Diabetes:

In [49]:
from sklearn.datasets import fetch_california_housing, load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

for loader in (fetch_california_housing, load_diabetes):
    Xi, yi = loader(return_X_y=True)
    Xi_train, Xi_test, yi_train, yi_test = train_test_split(Xi, yi, random_state=0)

    ols_lr = OLSLinearRegression()
    lr = LinearRegression()

    ols_lr.fit(Xi_train, yi_train)
    lr.fit(Xi_train, yi_train)

    ols_lr_preds = ols_lr.predict(Xi_test)
    lr_preds = lr.predict(Xi_test)
    np.testing.assert_array_almost_equal(ols_lr_preds, lr_preds, decimal=3, err_msg='Las predicciones difieren demasiado')

    ols_lr_score = ols_lr.score(Xi_test, yi_test)
    lr_score = lr.score(Xi_test, yi_test)
    print(loader.__name__ + ':\n\tR2 OLS: ' + str(ols_lr_score) + '\n\tR2 LS: ' + str(lr_score))
    np.testing.assert_almost_equal(ols_lr_score, lr_score, decimal=2, err_msg='Los scores difieren demasiado')

fetch_california_housing:
	R2 OLS: 0.5911695436406857
	R2 LS: 0.5911695436410476
load_diabetes:
	R2 OLS: 0.35940880381777063
	R2 LS: 0.3594088038177712
